In [3]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


def read_data(flux_path, label_path, batch_size):
    print("Loading flux and labels from npy files...")
    flux0 = np.load(flux_path).astype(np.float32)  
    labels = np.load(label_path, allow_pickle=True)
    
    print('Flux shape:', flux0.shape)
    print('Labels shape:', labels.shape)
    
    # 处理标签
    if len(labels.shape) == 1 or (len(labels.shape) == 2 and labels.shape[1] == 1):
        labels = labels.astype(np.int64).flatten()
        print("Classes:", np.unique(labels))
    
    # 填补缺失值
    from sklearn.impute import SimpleImputer
    flux_imp = SimpleImputer(missing_values=np.nan, strategy='median')
    flux = flux_imp.fit_transform(flux0)
    
    # 标准化特征
    scaler = StandardScaler()
    flux = scaler.fit_transform(flux)
    
    # 数据分割
    flux_train, flux_test, label_train, label_test = train_test_split(
        flux, labels, test_size=0.2, random_state=42, stratify=labels  # 分层抽样
    )
    
    # 为LSTM重塑数据
    seq_len = 25  # 设置序列长度
    feature_size = flux.shape[1] // seq_len  # 每个时间步的特征数
    
    flux_train_reshaped = []
    for i in range(flux_train.shape[0]):
        seq = []
        for j in range(seq_len):
            # 处理最后一段可能不足的情况
            start_idx = j * feature_size
            end_idx = min((j + 1) * feature_size, flux.shape[1])
            seq.append(flux_train[i, start_idx:end_idx])
        flux_train_reshaped.append(seq)
    
    flux_test_reshaped = []
    for i in range(flux_test.shape[0]):
        seq = []
        for j in range(seq_len):
            start_idx = j * feature_size
            end_idx = min((j + 1) * feature_size, flux.shape[1])
            seq.append(flux_test[i, start_idx:end_idx])
        flux_test_reshaped.append(seq)
    
    flux_train = np.array(flux_train_reshaped)
    flux_test = np.array(flux_test_reshaped)
    
    # 转换为PyTorch张量
    tensor_x_train = torch.from_numpy(flux_train)
    tensor_y_train = torch.from_numpy(label_train)
    tensor_x_test = torch.from_numpy(flux_test)
    tensor_y_test = torch.from_numpy(label_test)
    
    # 创建数据加载器
    train_dataset = Data.TensorDataset(tensor_x_train, tensor_y_train)
    test_dataset = Data.TensorDataset(tensor_x_test, tensor_y_test)
    
    train_loader = Data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True  # 确保随机洗牌
    )
    test_loader = Data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader


# 定义LSTM模型
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, output_size=3, dropout=0.3):
        super(LSTM_Model, self).__init__()
        
        # 添加dropout
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,  # 添加dropout防止过拟合
            bidirectional=True  # 使用双向LSTM
        )

        # 考虑双向LSTM的输出维度
        lstm_output_size = hidden_size * 2
        
        self.fc = nn.Sequential(
            nn.Linear(lstm_output_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),  # 添加BatchNorm
            nn.Dropout(dropout),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        # 初始化隐藏状态
        h0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(x.device)
        
        # LSTM前向传播
        output, (h_n, c_n) = self.lstm(x, (h0, c0))
        
        # 取最后一个时间步
        out = output[:, -1, :]
        out = self.fc(out)
        return out


# 训练函数
def train_and_evaluate(model, train_loader, test_loader, device, num_epochs=100, learning_rate=0.001):

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    best_accuracy = 0
    
    for epoch in range(num_epochs):
        # 训练阶段
        model.train()
        train_loss = 0.0
        
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            
            # 梯度裁剪，防止梯度爆炸
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
        
        # 计算平均训练损失
        train_loss = train_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}")
        
        if (epoch + 1) % 10 == 0:
            accuracy = evaluate(model, test_loader, device)
            scheduler.step(train_loss)
            
            # 保存模型
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                torch.save(model.state_dict(), 'best_lstm_model.pth')
    
    return model


# 评估函数
def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            _, predicted = torch.max(outputs.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    
    accuracy = correct / total
    print(f"Accuracy: {accuracy:.4f}")
    return accuracy


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    
    torch.manual_seed(42)
    np.random.seed(42)
    
    device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    num_epochs = 100
    batch_size = 64
    learning_rate = 0.001
    
    flux_path = './flux.npy'
    label_path = './spectypes.npy'
    
    train_loader, test_loader = read_data(flux_path, label_path, batch_size)
    
    for x_batch, _ in train_loader:
        input_size = x_batch.shape[2]  # [batch_size, seq_len, input_size]
        break
    

    model = LSTM_Model(input_size=input_size).to(device)
    train_and_evaluate(model, train_loader, test_loader, device, num_epochs, learning_rate)

Using device: cuda:7
Loading flux and labels from npy files...
Flux shape: (6000, 7781)
Labels shape: (6000,)
Classes: [0 1 2]
Epoch [1/100], Train Loss: 0.6896
Epoch [2/100], Train Loss: 0.6535
Epoch [3/100], Train Loss: 0.5848
Epoch [4/100], Train Loss: 0.5253
Epoch [5/100], Train Loss: 0.4947
Epoch [6/100], Train Loss: 0.4768
Epoch [7/100], Train Loss: 0.4630
Epoch [8/100], Train Loss: 0.4484
Epoch [9/100], Train Loss: 0.4199
Epoch [10/100], Train Loss: 0.4055
Accuracy: 0.8325
Epoch [11/100], Train Loss: 0.3836
Epoch [12/100], Train Loss: 0.3751
Epoch [13/100], Train Loss: 0.3377
Epoch [14/100], Train Loss: 0.3341
Epoch [15/100], Train Loss: 0.3220
Epoch [16/100], Train Loss: 0.3111
Epoch [17/100], Train Loss: 0.2868
Epoch [18/100], Train Loss: 0.2611
Epoch [19/100], Train Loss: 0.2460
Epoch [20/100], Train Loss: 0.2321
Accuracy: 0.8500
Epoch [21/100], Train Loss: 0.2425
Epoch [22/100], Train Loss: 0.2432
Epoch [23/100], Train Loss: 0.2172
Epoch [24/100], Train Loss: 0.2103
Epoch [2